In [1]:
import re
import json
import pandas as pd
from glob import glob

In [2]:
AD_KEYWORDS = [
    "일부내돈내산",
    "지원을받아",
    "지원받아",
    "후원을받아",
    "후원받아",
    "원고료",
    "체험단",
    "제공받아",
    "제공받았",
    "제공서비스",
    "금전적지원",
    "사비로",
    "업체로부터",
    "협찬",
    "제품또는서비스를",
    "업체에서",
    "소정의",
    "고료를",
    "할인을",
    "레뷰",
    "강남오빠",
    "디너의여왕",
    "서울오빠",
    "이놀자데뷰"
]

HAN_REGEX = re.compile('[^가-힣]')

In [3]:
files = glob('./data/semi_struct_ocr/*.json')
files

['./data/semi_struct_ocr/홍대 회식 맛집.json',
 './data/semi_struct_ocr/홍대 데이트 맛집.json',
 './data/semi_struct_ocr/성수 데이트 맛집.json',
 './data/semi_struct_ocr/압구정 회식 맛집.json',
 './data/semi_struct_ocr/강남역 데이트 맛집.json',
 './data/semi_struct_ocr/영등포 회식 맛집.json',
 './data/semi_struct_ocr/성수 회식 맛집.json',
 './data/semi_struct_ocr/압구정 데이트 맛집.json',
 './data/semi_struct_ocr/강남역 회식 맛집.json',
 './data/semi_struct_ocr/범계 회식 맛집.json',
 './data/semi_struct_ocr/범계 데이트 맛집.json',
 './data/semi_struct_ocr/영등포 데이트 맛집.json']

In [4]:
def detect_ad_words(text: str) -> bool:
    if not text: return False
    
    text = HAN_REGEX.sub('', text)
    for kywd in AD_KEYWORDS:
        if kywd in text:
            return True
    return False
    
def number_text_pairwise(contents: list[dict], labeling=True) -> dict:
    result = {}
    labels = []
    for content in contents:
        text = content.get('ocr_text', content['content'])
        result[content['no']] = text
        
        if detect_ad_words(text): labels.append(content['no'])
    
    if labeling:
        return (result, labels)
    
    return result


def make_dataset(contents: dict, labels: list):
    result = []
    for k, v in contents.items():
        if k in labels:
            result.append(
                (v, 1)
            )
        else:
            result.append((v, 0))
            
    return result

In [5]:
result = []
for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        blog = json.load(f)
    
    for post in blog:
        content, labels = number_text_pairwise(post['contents'], labeling=True)
        result += make_dataset(content, labels)

In [6]:
df = pd.DataFrame(result, columns=['text', 'label'])
df

,text,label
0,홍대에서 데이트 하기로 한 주말\n진짜 간만에 홍대\n전에 항상 세우던 공영주차장이...,0
1,Vied INTF 주택상업공간 HAN SALSN 쟁 반 집 反 절염,0
2,오픈전 지나갈때는 사람이 없었는데\n오픈시간 12시가 다가오면 #홍대고기집맛집 앞에...,0
3,재반집 JAENG BAN JIB8292 OPEN12:00 PM CLOSE 02:00...,0
4,나도 홍대 #고기집 쟁반집 앞에서 기다려야지!,0
...,...,...
639533,,0
639534,,0
639535,은근 자리도 넓고 안주도 저렴한데 맛있고 하이볼 종류도 많아서 내가 조와하는곳이징 ...,0
639536,9! 월마감날 함께한 블랙그레이즈드라떼 가을이왔구나 !,0


In [11]:
df.to_csv('./data/cost_detect_dataset2.csv', encoding='utf-8')

In [7]:
df = pd.read_csv('./data/cost_detect_dataset.csv', index_col=0).fillna('')
df

,text,label
0,홍대에서 데이트 하기로 한 주말\n진짜 간만에 홍대\n전에 항상 세우던 공영주차장이...,0
1,Vied INTF 주택상업공간 HAN SALSN 쟁 반 집 反 절염,0
2,오픈전 지나갈때는 사람이 없었는데\n오픈시간 12시가 다가오면 #홍대고기집맛집 앞에...,0
3,재반집 JAENG BAN JIB8292 OPEN12:00 PM CLOSE 02:00...,0
4,나도 홍대 #고기집 쟁반집 앞에서 기다려야지!,0
...,...,...
639533,,0
639534,,0
639535,은근 자리도 넓고 안주도 저렴한데 맛있고 하이볼 종류도 많아서 내가 조와하는곳이징 ...,0
639536,9! 월마감날 함께한 블랙그레이즈드라떼 가을이왔구나 !,0


In [8]:
import re

In [9]:
df['text'] = df['text'].apply(lambda x: re.sub("[^a-zA-Z가-힣]", '' ,x))

In [10]:
df

,text,label
0,홍대에서데이트하기로한주말진짜간만에홍대전에항상세우던공영주차장이막혀있어서근처유료주차장에...,0
1,ViedINTF주택상업공간HANSALSN쟁반집절염,0
2,오픈전지나갈때는사람이없었는데오픈시간시가다가오면홍대고기집맛집앞에기다리는사람들이하나둘생...,0
3,재반집JAENGBANJIBOPENPMCLOSEAMWIFNOPETTOILETDELIV...,0
4,나도홍대고기집쟁반집앞에서기다려야지,0
...,...,...
639533,,0
639534,,0
639535,은근자리도넓고안주도저렴한데맛있고하이볼종류도많아서내가조와하는곳이징팝콘옥수수꼭주문해주세...,0
639536,월마감날함께한블랙그레이즈드라떼가을이왔구나,0
